In [1]:
import csv
import pickle
import numpy as np
import torch
import pprint
from transformers import BertModel, BertTokenizer

In [2]:
#Define File Path
data = 'tw_charitynews_4bert.csv'

In [3]:
news_content = []
stance_label = []

In [4]:
with open(data, 'r', encoding='utf-8') as file:
    reader = csv.reader(file)
    next(reader)  # 跳過標題行
    for row in reader:
        string = row[1]
        label = row[2]
        string_list = string.split(' ')  # 以空格分割字串
        news_content.append(string_list)
        stance_label.append(label)

In [5]:
pprint.pprint(news_content)

[['一',
  '名',
  '雙眼',
  '失明',
  '闕',
  '小弟',
  '父親',
  '入獄',
  '肢體',
  '障礙',
  '母親',
  '遠',
  '高雄',
  '剩',
  '弟弟',
  '2',
  '人',
  '北部',
  '生活',
  '揹',
  '一',
  '大',
  '籃',
  '糖果',
  '街',
  '上',
  '兜售',
  '經過',
  '好心',
  '網友',
  '網路',
  '上',
  '幫忙',
  '宣傳',
  '吸引',
  '民眾',
  '愛心',
  '湧入',
  '網傳',
  '許多',
  '資訊',
  '過舊',
  '新北市',
  '勞工局',
  '主動',
  '說明',
  '闕',
  '小弟',
  '現況',
  '勞工局',
  '表示',
  '政府',
  '單位',
  '101年',
  '10月',
  '受理',
  '小偉',
  '就業',
  '輔導',
  '職業',
  '重建',
  '管理員',
  '評估',
  '後',
  '認為',
  '闕',
  '小弟',
  '能力',
  '適合',
  '職場',
  '有意',
  '推介',
  '愛盲',
  '庇護工場',
  '協助',
  '建立',
  '工作',
  '態度',
  '技能',
  '表態',
  '願意',
  '102年',
  '2月',
  '職管員',
  '安排',
  '視障者',
  '有聲書',
  '生活',
  '訓練',
  '協助',
  '適應',
  '未來',
  '職場',
  '103年',
  '安排',
  '擔任',
  '愛盲',
  '庇護工場',
  '見習生',
  '段',
  '期間',
  '開始',
  '利用',
  '下班',
  '後',
  '販賣',
  '口香糖',
  '103年',
  '10月',
  '自願',
  '離開',
  '愛盲',
  '庇護工場',
  '外界',
  '闕',
  '小弟',
  '當時',
  '庇護工場',
  '工作',
  '月薪',
  '800',
  '元',
  '疑慮',
  '勞工局',


In [6]:
print(stance_label)

['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '1', '1', '1', '1', '1', '0', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '2', '2', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '0', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '2', '1', '2', '1', '2', '1', '2', '1', '2', '2', '1', '2', '2', '2', '1', '1', '2', '1', '1', '1', '1', '1', '2', '1', '1', '2', '2', '1', '1', '1', '1', '1', '2', '2', '2', '2', '2', '2', '1', '1', '1', '1', '1', '2', '2', '2', '2', '2', '2', '2', '2', '1', '1', '1', '1', '1', '2', '2', '1', '2', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '2', '1', '2', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '2', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '2', '1', '1', '2', '1', '1', '1', '2', '2', '2', '2', '1', '1', '2', '1', '1',

In [7]:
model_name = 'bert-base-chinese'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
def process_text(text):
    tokens = ['[CLS]'] + text + ['[SEP]']
    if len(tokens) > 512:
        tokens = tokens[:512]  # 截断超过512个tokens的文本
    else:
        tokens = tokens + ['[PAD]'] * (512 - len(tokens))  # 填充不足512个tokens的文本
    
    input_ids = tokenizer.convert_tokens_to_ids(tokens)
    return input_ids

def convert_to_vectors(texts):
    vectors = []
    for text in texts:
        input_ids = process_text(text)
        input_tensor = torch.tensor([input_ids])

        with torch.no_grad():
            outputs = model(input_tensor)
        
        sentence_vector = outputs.last_hidden_state.mean(dim=1).squeeze(0)
        vectors.append(sentence_vector)
    return vectors

In [12]:
vectors = convert_to_vectors(news_content)

In [14]:
print(vectors[0].shape)

torch.Size([512, 768])


In [15]:
print(len(vectors))

445


In [22]:
with open('twc_news_vec.pkl', 'wb') as f:
    pickle.dump(vectors, f)

In [23]:
vectors_np = np.array([vector.numpy() for vector in vectors])
np.savez('twc_news_vec.npz', vectors=vectors_np)